<a href="https://colab.research.google.com/github/balu1866/TeluguLLM/blob/main/Telugu_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Langchain
!pip install datasets
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [2]:
import pandas as pd
import random

df = pd.read_csv("hf://datasets/Telugu-LLM-Labs/telugu_alpaca_yahma_cleaned_filtered_romanized/yahma_alpaca_cleaned_telugu_filtered_and_romanized.csv")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
df = df[['instruction', 'output', 'telugu_instruction', 'telugu_output', 'telugu_transliterated_instruction', 'telugu_transliterated_output']]
print(df.columns)


Index(['instruction', 'output', 'telugu_instruction', 'telugu_output',
       'telugu_transliterated_instruction', 'telugu_transliterated_output'],
      dtype='object')


In [4]:

import json

eng_tel_map = {}
eng_tel_list =[]

eng_tel_trans_map = {}
eng_tel_trans_list =[]

tel_trans_eng_map = {}
tel_trans_eng_list =[]

tel_trans_tel_map = {}
tel_trans_tel_list =[]

tel_trans_tel_trans_map = {}
tel_trans_tel_trans_list =[]

tel_tel_trans_map = {}
tel_tel_trans_list =[]

tel_eng_map = {}
tel_eng_list =[]

tel_tel_map = {}
tel_tel_list =[]


for index, row in df.iterrows():
  # Access individual columns using their names
  instruction = row['instruction']
  output = row['output']
  telugu_instruction = row['telugu_instruction']
  telugu_output = row['telugu_output']
  telugu_transliterated_instruction = row['telugu_transliterated_instruction']
  telugu_transliterated_output = row['telugu_transliterated_output']

  # Telugu output
  eng_tel_list.append({
      'english_instruction': instruction,
      'telugu_output': telugu_output,
  })

  tel_tel_list.append({
      'telugu_instruction': telugu_instruction,
      'telugu_output': telugu_output,
  })

  tel_trans_tel_list.append({
      'telugu_transliterated_instruction': telugu_transliterated_instruction,
      'telugu_output': telugu_output,
  })

  # Telugu transliterated output
  eng_tel_trans_list.append({
      'english_instruction': instruction,
      'telugu_transliterated_output': telugu_transliterated_output,
  })

  tel_tel_trans_list.append({
      'telugu_instruction': telugu_instruction,
      'telugu_transliterated_output': telugu_transliterated_output,
  })

  tel_trans_tel_trans_list.append({
      'telugu_transliterated_instruction': telugu_transliterated_instruction,
      'telugu_transliterated_output': telugu_transliterated_output,
  })

  # English output
  tel_eng_list.append({
      'telugu_instruction': telugu_instruction,
      'english_output': output,
  })

  tel_trans_eng_list.append({
      'telugu_transliterated_instruction': telugu_transliterated_instruction,
      'english_output': output,
  })


dataset_list = []
dataset_list.extend(eng_tel_list)
dataset_list.extend(tel_tel_list)
dataset_list.extend(tel_trans_tel_list)

dataset_list.extend(eng_tel_trans_list)
dataset_list.extend(tel_tel_trans_list)
dataset_list.extend(tel_trans_tel_trans_list)

dataset_list.extend(tel_eng_list)
dataset_list.extend(tel_trans_eng_list)

random.shuffle(dataset_list)


In [5]:
def format_record(example):
    if 'english_instruction' in example and 'telugu_output' in example:
        return {
            "prompt": f"Translate the following instruction to Telugu:\n{example['english_instruction']}",
            "completion": example["telugu_output"]
        }

    elif 'telugu_instruction' in example and 'telugu_output' in example:
        return {
            "prompt": f"Rewrite the following in Telugu:\n{example['telugu_instruction']}",
            "completion": example["telugu_output"]
        }

    elif 'telugu_transliterated_instruction' in example and 'telugu_output' in example:
        return {
            "prompt": f"Translate this transliterated instruction to Telugu:\n{example['telugu_transliterated_instruction']}",
            "completion": example["telugu_output"]
        }

    elif 'english_instruction' in example and 'telugu_transliterated_output' in example:
        return {
            "prompt": f"Translate to Telugu (transliterated):\n{example['english_instruction']}",
            "completion": example["telugu_transliterated_output"]
        }

    elif 'telugu_instruction' in example and 'telugu_transliterated_output' in example:
        return {
            "prompt": f"Transliterate the following Telugu instruction:\n{example['telugu_instruction']}",
            "completion": example["telugu_transliterated_output"]
        }


    elif 'telugu_transliterated_instruction' in example and 'telugu_transliterated_output' in example:
        return {
            "prompt": f"Complete this transliterated Telugu instruction:\n{example['telugu_transliterated_instruction']}",
            "completion": example["telugu_transliterated_output"]
        }

    elif 'telugu_instruction' in example and 'english_output' in example:
        return {
            "prompt": f"Translate this Telugu instruction to English:\n{example['telugu_instruction']}",
            "completion": example["english_output"]
        }

    elif 'telugu_transliterated_instruction' in example and 'english_output' in example:
        return {
            "prompt": f"Translate this transliterated Telugu instruction to English:\n{example['telugu_transliterated_instruction']}",
            "completion": example["english_output"]
        }

    else:
        return None

formatted_data = []

for item in dataset_list:
    formatted = format_record(item)
    if formatted:  # skip malformed
        formatted_data.append(formatted)

with open('dataset.json', 'w', encoding='utf-8') as file:
  for item in formatted_data:
    file.write(json.dumps(item) + '\n')



In [6]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `bluepersonal1866` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `blueper

In [7]:
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "mistralai/Mistral-7B-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map="auto")

model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
model = get_peft_model(model, peft_config)



tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
ERROR:bitsandbytes.cextension:Could not load bitsandbytes native library: /usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: cannot open shared object file: No such file or directory
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 85, in <module>
    lib = get_native_library()
          ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 72, in get_native_library
    dll = ct.cdll.LoadLibrary(str(binary_path))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 454, in LoadLibrary
    return self._dlltype(name)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in _

RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
from datasets import load_dataset, Dataset

def tokenize(record):
  full_prompt = record['prompt'] + "\n" + record['completion']
  tokenized = tokenizer(
      full_prompt,
      max_length=512,
      padding="max_length",
      truncation=True,
      return_tensors="pt"
  )
  prompt_ids = tokenizer(record['prompt'], truncation=True, max_length=512)['input_ids']
  prompt_ids_len = len(prompt_ids)
  labels = tokenized['labels'].copy()
  labels[:prompt_ids_len] = [-100] * prompt_ids_len
  tokenized['lables'] = labels
  return tokenized

# dataset = Dataset.from_list(formatted_data)
dataset = load_dataset("json", data_files="dataset.json")

split_dataset = dataset["train"].train_test_split(test_size=0.2)

train_dataset = split_dataset["train"]
temp_dataset = split_dataset["test"].train_test_split(test_size=0.5)

val_dataset = temp_dataset["train"]
test_dataset = temp_dataset["test"]

tokenized_train = train_dataset.map(tokenize, remove_columns=dataset.column_names)
tokenized_val = val_dataset.map(tokenize, remove_columns=dataset.column_names)
tokenized_test = test_dataset.map(tokenize, remove_columns=dataset.column_names)


print(tokenized_train[0])
